In [1]:
%pip install scikit-fuzzy

     ---------------------------------------- 0.0/994.0 kB ? eta -:--:--
     - ------------------------------------- 30.7/994.0 kB 1.4 MB/s eta 0:00:01
     -- ---------------------------------- 61.4/994.0 kB 656.4 kB/s eta 0:00:02
     ---- ------------------------------- 112.6/994.0 kB 939.4 kB/s eta 0:00:01
     ------- ------------------------------ 204.8/994.0 kB 1.1 MB/s eta 0:00:01
     --------- ---------------------------- 256.0/994.0 kB 1.1 MB/s eta 0:00:01
     ------------------- ------------------ 501.8/994.0 kB 1.9 MB/s eta 0:00:01
     ---------------------------- --------- 737.3/994.0 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 994.0/994.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (py


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import numpy as np